In [1]:
%pip install pinecone
%pip install llama-cpp-python
%pip install chromadb


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install spacy --only-binary=:all:


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
!python -m spacy download en_core_web_sm


c:\Users\ashaa\AppData\Local\Programs\Python\Python313\python.exe: No module named spacy


In [ ]:
%pip install -U spacy
%pip install -U spacy-transformers
!python -m spacy download en_core_web_trf


In [ ]:
%pip install -qU \
  sentence-transformers==2.2.2 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -qU \
    openai==0.27.7 \


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -qU \
    langchain==0.1.1 \
    langchain-community==0.0.13 \
    tiktoken 

^C
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [1]:
import spacy

# Load the more powerful transformer-based model
nlp = spacy.load("en_core_web_trf")

latest_org = None  # Variable to store the latest company

def extract_organization_spacy(question:str):
    global latest_org  

    doc = nlp(question)
    # Extract entities labeled as "ORG" (organizations)
    orgs = [ent.text for ent in doc.ents if ent.label_ == "ORG"]

    if orgs:
        latest_org = orgs[0]  # Update with the detected organization
    print(orgs)
    return orgs

In [2]:
question3 = "tell me about accenture and modak".title()
print(extract_organization_spacy(question3)) 
question3

['Accenture', 'Modak']
['Accenture', 'Modak']


'Tell Me About Accenture And Modak'

In [3]:
import json
with open('data.json', 'r',encoding='utf-8') as file:
    data = json.load(file)
# f=open("data.json","r",encoding='utf-8')
ii=0
for i in data:
    ii+=1
    if i["id"]=="realpage_description":
        print(i,ii)

{'id': 'realpage_description', 'text': "RealPage is a leading global provider of property management software solutions for the real estate industry. Headquartered in Richardson, Texas, RealPage India, located in Hyderabad, plays a crucial role in the company's global operations. The company offers innovative software platforms that help property managers and owners optimize their operations, improve tenant experiences, and maximize revenue. RealPage is known for its expertise in SaaS (Software as a Service) solutions, data analytics, and artificial intelligence (AI) in the real estate sector."} 300


In [121]:
data[0]["id"]

'accenture_description'

In [188]:
batched_id_data=[]
for i in range(0,len(data),96):
    a=[]
    for j in data[i:i+96]:
        a.append(j["id"])
    batched_id_data.append(a)
# batched_id_data

In [189]:
batched_data=[]
for i in range(0,len(data),96):
    batched_data.append(data[i:i+96])
# batched_data

In [86]:
len(batched_data)

7

In [87]:
for i in batched_data:
    print(len(i),i)

96 [{'id': 'accenture_description', 'text': 'Accenture is a global professional services company that specializes in digital transformation, cloud computing, artificial intelligence, and consulting solutions. Headquartered in Dublin, Ireland, it operates in over 120 countries with approximately 743,000 employees, serving 75% of Fortune Global 500 companies. Accenture helps businesses, governments, and organizations optimize operations, accelerate growth, and enhance customer experiences through cutting-edge technologies. The company offers services across five key areas: Strategy & Consulting, Technology, Operations, Industry X, and Song, enabling clients to drive innovation and achieve sustainable success. With strong partnerships with technology leaders like Microsoft, Google, AWS, SAP, and Oracle, Accenture delivers high-impact solutions tailored to industry needs. It is also committed to sustainability, diversity, and corporate social responsibility, striving for net-zero emissions

In [ ]:
from pinecone import  Pinecone

api_key = "APIKEY"

pc = Pinecone(api_key=api_key)

from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)


In [136]:
pc.delete_index("example")

In [98]:
batched_embeddings=[]
for i in batched_data:
    embeddings = pc.inference.embed(
        model="multilingual-e5-large",
        inputs=[d["text"] for d in i],
        parameters={
            "input_type": "passage", 
            "truncate": "END"
        }
    )
    batched_embeddings.append(embeddings)

print(batched_embeddings)
# da

# embeddings = pc.inference.embed(
#         model="multilingual-e5-large",
#         inputs=[d["text"] for d in data],
#         parameters={
#             "input_type": "passage", 
#             "truncate": "END"
#         }
#     )

# print(embeddings)


[EmbeddingsList(
  model='multilingual-e5-large',
  vector_type='dense',
  data=[
    {'vector_type': dense, 'values': [0.0014705657958984375, 0.00334930419921875, ..., -0.0237884521484375, -0.006641387939453125]},
    {'vector_type': dense, 'values': [0.04315185546875, -0.0220489501953125, ..., -0.01593017578125, -0.007568359375]},
    ... (92 more embeddings) ...,
    {'vector_type': dense, 'values': [0.023590087890625, -0.03179931640625, ..., -0.039581298828125, 0.0009250640869140625]},
    {'vector_type': dense, 'values': [0.037261962890625, -0.045867919921875, ..., -0.03106689453125, 0.00627899169921875]}
  ],
  usage={'total_tokens': 10908}
), EmbeddingsList(
  model='multilingual-e5-large',
  vector_type='dense',
  data=[
    {'vector_type': dense, 'values': [0.028961181640625, -0.049896240234375, ..., -0.0173797607421875, 0.0089569091796875]},
    {'vector_type': dense, 'values': [0.0258331298828125, -0.042266845703125, ..., -0.018951416015625, -0.00556182861328125]},
    ... (

In [4]:
query = "tell me about accenture package"
# Convert the query into a numerical vector that Pinecone can search with
query_embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

In [20]:
import numpy as np
def cosine(a,b):
    dot_product = np.dot(a,b)
    norm_vector1 = np.linalg.norm(a)
    norm_vector2 = np.linalg.norm(b)
    cosine_similarity = dot_product / (norm_vector1 * norm_vector2)
    return cosine_similarity


In [15]:
a=query_embedding.data[0].values

In [18]:
b=embeddings.data[0].values

In [28]:
metrics=[]
for i in embeddings.data:
    metrics.append({"distance":cosine(i.values,a),})
# metrics

In [8]:
batched_embeddings[0].data[0]

NameError: name 'batched_embeddings' is not defined

In [2]:
index_name = "example"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws", 
            region="us-east-1"
        ) 
        
    ) 

while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)


In [3]:
index = pc.Index(index_name)


In [115]:
batched_embeddings[0]

EmbeddingsList(
  model='multilingual-e5-large',
  vector_type='dense',
  data=[
    {'vector_type': dense, 'values': [0.0014705657958984375, 0.00334930419921875, ..., -0.0237884521484375, -0.006641387939453125]},
    {'vector_type': dense, 'values': [0.04315185546875, -0.0220489501953125, ..., -0.01593017578125, -0.007568359375]},
    ... (92 more embeddings) ...,
    {'vector_type': dense, 'values': [0.023590087890625, -0.03179931640625, ..., -0.039581298828125, 0.0009250640869140625]},
    {'vector_type': dense, 'values': [0.037261962890625, -0.045867919921875, ..., -0.03106689453125, 0.00627899169921875]}
  ],
  usage={'total_tokens': 10908}
)

In [133]:

records = []
# for i in batched_embeddings:
#     print(i[0])    
#     for d, e in zip(data, i):
#         # print(d["id"])
#         records.append({
#             "id": d["id"],
#             "values": e["values"],
#             "metadata": {
#                 "source_text": d["text"]
#             }
#         })
c=0
for i in batched_embeddings:
    for j in i:
        records.append({
            "id":data[c]["id"],
            "values":j["values"],
            "metadata":{
                "source_text":data[c]["text"]
            }
        })
        c+=1
# Upsert the records into the index
# index.upsert(
#     vectors=records,
#     namespace="example"
# )

# for batch in batched_embeddings:
    # for d, e in zip(data, batch.data):
    #     records = []
    #     records.append({
    #         "id": d["id"],
    #         "values": e["values"],
    #         "metadata": {
    #             "source_text": d["text"]
    #         }
    #     })
    
    # # Upsert the current batch
    #     index.upsert(
    #         vectors=records,
    #         namespace="example"
    #     )
        



# records = []
# for d, e in zip(data, embeddings):
#     records.append({
#         "id": d["id"],
#         "values": e["values"],
#         "metadata": {
#             "source_text": d["text"]
#         }
#     })
# # Upsert the records into the index
# index.upsert(
#         vectors=records,
#         namespace="example"
#     )


In [190]:
# for i  in records:
    # print(i["id"])

In [139]:
index.upsert(
    vectors=records[:96],
    namespace="example0"
)


{'upserted_count': 96}

In [140]:
index.upsert(
    vectors=records[96*1:96*2],
    namespace="example1"
)


{'upserted_count': 96}

In [141]:
index.upsert(
    vectors=records[96*2:96*3],
    namespace="example2"
)

{'upserted_count': 96}

In [142]:
index.upsert(
    vectors=records[96*3:96*4],
    namespace="example3"
)

{'upserted_count': 96}

In [143]:
index.upsert(
    vectors=records[96*4:96*5],
    namespace="example4"
)

{'upserted_count': 96}

In [144]:
index.upsert(
    vectors=records[96*5:96*6],
    namespace="example5"
)

{'upserted_count': 96}

In [145]:
index.upsert(
    vectors=records[96*6:],
    namespace="example6"
)

{'upserted_count': 52}

In [150]:
print(index.describe_index_stats())


{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'example0': {'vector_count': 96},
                'example1': {'vector_count': 96},
                'example2': {'vector_count': 94},
                'example3': {'vector_count': 96},
                'example4': {'vector_count': 96},
                'example5': {'vector_count': 96},
                'example6': {'vector_count': 52}},
 'total_vector_count': 626,
 'vector_type': 'dense'}


In [148]:
# a=[]
# print(data)
# for i in data:
#     a.append(index.fetch(ids=[i["id"]], namespace="example"))
# a

In [51]:
# a

In [74]:
index.fetch(ids=["accenture_description"], namespace="example2")


FetchResponse(namespace='example2', vectors={'accenture_description': Vector(id='accenture_description', values=[0.0277557373, -0.0253448486, -0.0421142578, -0.0440063477, 0.0270996094, -0.0309448242, -0.0363464355, 0.0534973145, 0.0662841797, -0.0302886963, 0.0366210938, 0.0219421387, -0.0553588867, 0.0157318115, -0.0294952393, 0.000319719315, -0.00894928, 0.051574707, -0.0190887451, -0.00929260254, 0.0399169922, -0.0219268799, -0.0188751221, -0.0210571289, -0.0469665527, -0.0185241699, -0.0488891602, -0.0391845703, -0.00644683838, -0.0117416382, 0.00338172913, 0.00405502319, -0.0496826172, -0.0390625, -0.0354309082, 0.041015625, 0.0376281738, 0.0254058838, -0.0243225098, 0.0452575684, -0.0292816162, 0.0281524658, -0.0210723877, -0.0391845703, -0.0263061523, 0.0215759277, 0.0545349121, -0.0245513916, -0.0265197754, 0.011177063, 0.0261688232, 0.0194396973, -0.013999939, -0.0267486572, -0.0104751587, 0.048248291, -0.0148849487, 0.0046081543, -0.0377502441, -0.032043457, -0.00396347046, 

In [7]:
# Define your query
query="1) roles offered by realpage\n     2) roles offered by razor pay"
query = "tell me about accenture package"
query2="package of sde roles"
query3="roles offered by factset"
query4="Is python necessary for full stack roles"
company_name=(extract_organization_spacy(query))
# Convert the query into a numerical vector that Pinecone can search with
query_embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

res=[]
for i in range(7):
    results = index.query(
    namespace=f"example{i}",
    vector=query_embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
    )
    res.append(results)
# Search the index for the three most similar vectors
results = index.query(
    namespace="example",
    vector=query_embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)
print(results['matches'])

doc=[]
for i in results['matches']:
    doc.append({"id":i['id'],"source_text":i['metadata']['source_text']})



[]
[]


In [8]:
doc=[]
for j in res:
    for i in j['matches']:
        doc.append({"id":i['id'],"source_text":i['metadata']['source_text']})
# doc

In [9]:
res

[{'matches': [{'id': 'accenture_role_description',
               'metadata': {'source_text': 'Accenture Role description: '
                                           'Packaged App development associate '
                                           'Join our team in developing, '
                                           'designing, and maintaining '
                                           'technologies that enhance the way '
                                           'our clients and the world operate. '
                                           'You will work in challenging and '
                                           'dynamic environments, leveraging '
                                           'your versatility to create and '
                                           'support technology solutions that '
                                           'align with client requirements, '
                                           'from analysis to implementation. As '
          

In [10]:
# Rerank the search results based on their relevance to the query
ranked_results = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query=query,
    documents=doc,
    top_n=5,
    rank_fields=["source_text"],
    return_documents=True,
    parameters={
        "truncate": "END"
    }
)

print(str(ranked_results))


RerankResult(
  model='bge-reranker-v2-m3',
  data=[{
    index=1,
    score=0.81960475,
    document={
        id='accenture_additional_info',
        source_text='Accenture offers two roles with different salary packages. The Associate Software Engineer role comes with a package of Rs. 4.50 LPA, which includes a fixed pay of Rs. 3.83 LPA, a performance bonus of Rs. 32,500 per annum, a one-time joining bonus of Rs. 25,000, along with additional benefits such as gratuity and insurance. The Advanced Associate Software Engineer role offers a Rs. 6.50 LPA package, with a fixed pay of Rs. 5.41 LPA, a performance-based pay of Rs. 82,500 per performance cycle, and a one-time joining bonus of Rs. 26,500. Employees also receive additional benefits such as free transport, house rent allowance, medical allowance, and health insurance with a Rs. 4 lakh cover, which is an advantage over competitors like TCS, CTS, Infosys, and Wipro.'
    }
  },{
    index=2,
    score=0.43512127,
    document={
  

In [11]:
context=""
for i in ranked_results.data:
    if i.score>=0.3:
        context+=i['document']['source_text']+". "   
context

'Accenture offers two roles with different salary packages. The Associate Software Engineer role comes with a package of Rs. 4.50 LPA, which includes a fixed pay of Rs. 3.83 LPA, a performance bonus of Rs. 32,500 per annum, a one-time joining bonus of Rs. 25,000, along with additional benefits such as gratuity and insurance. The Advanced Associate Software Engineer role offers a Rs. 6.50 LPA package, with a fixed pay of Rs. 5.41 LPA, a performance-based pay of Rs. 82,500 per performance cycle, and a one-time joining bonus of Rs. 26,500. Employees also receive additional benefits such as free transport, house rent allowance, medical allowance, and health insurance with a Rs. 4 lakh cover, which is an advantage over competitors like TCS, CTS, Infosys, and Wipro.. Accenture is a global professional services company that specializes in digital transformation, cloud computing, artificial intelligence, and consulting solutions. Headquartered in Dublin, Ireland, it operates in over 120 countr

In [36]:
# context=(ranked_results.data[0]['document']['source_text'])
# print(context)

In [ ]:
APIKEY

In [ ]:
from huggingface_hub import login

login("APIKEY")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\ashaa\.cache\huggingface\token
Login successful


In [ ]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("text-generation", model="meta-llama/Meta-Llama-3-8B")

c:\Users\ashaa\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [03:58<00:00, 59.73s/it]
Device set to use cpu


In [ ]:
# response = pipe("Once upon a time, in a distant land,", max_length=100, do_sample=True)
# print(response)
# # 


In [4]:
from llama_cpp import Llama
llm=Llama(model_path=r"C:\Users\ashaa\Downloads\major\Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf")

llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from C:\Users\ashaa\Downloads\major\Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str              = 8B
llama_model_loader: - kv   6:                            general.license str              = llama3.1
llam

In [19]:
llm("hello?")

llama_perf_context_print:        load time =    1137.37 ms
llama_perf_context_print: prompt eval time =    1136.89 ms /     3 tokens (  378.96 ms per token,     2.64 tokens per second)
llama_perf_context_print:        eval time =    2611.78 ms /    15 runs   (  174.12 ms per token,     5.74 tokens per second)
llama_perf_context_print:       total time =    3769.90 ms /    18 tokens


{'id': 'cmpl-d8274b99-c1e7-46f2-8f0f-a0ff23e14037',
 'object': 'text_completion',
 'created': 1740100496,
 'model': 'C:\\Users\\ashaa\\Downloads\\major\\Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf',
 'choices': [{'text': " (I'm a little scared)\nI was wondering if you could help me with",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 3, 'completion_tokens': 16, 'total_tokens': 19}}

In [5]:
response = llm("What is machine learning? explain in 200 words",max_tokens=200)
# print(response["choices"][0]["text"])
print(response)

Llama.generate: 10 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    1248.49 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =   36685.95 ms /   200 runs   (  183.43 ms per token,     5.45 tokens per second)
llama_perf_context_print:       total time =   37037.81 ms /   201 tokens


{'id': 'cmpl-0fb11a66-596c-4655-8a4d-2de9653eb7fc', 'object': 'text_completion', 'created': 1739864599, 'model': 'C:\\Users\\ashaa\\Downloads\\major\\Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf', 'choices': [{'text': "\nMachine learning is a subset of artificial intelligence that enables systems to automatically learn and improve from experience without being explicitly programmed. It involves training algorithms on data to make predictions, classify objects, or make decisions.\nImagine you have a large dataset of images, and you want to build a system that can identify whether a picture is of a cat or a dog. You wouldn't explicitly program the system to recognize certain features; instead, you would train a machine learning algorithm on the data, and it would learn to recognize patterns and make predictions based on that training.\nThere are three types of machine learning:\n1. Supervised learning: The algorithm is trained on labeled data, and it learns to make predictions based on that da

In [10]:
response = llm(
    "Explain black holes in simple terms.",
    max_tokens=150,       # Maximum number of tokens to generate
    temperature=0.7,      # Higher = more random, lower = more deterministic
    top_k=50,             # Consider top 50 most likely tokens
    top_p=0.9,            # Nucleus sampling (higher = more diverse output)
    echo=False            # Do not repeat the input in output
)

print(response["choices"][0]["text"])


Llama.generate: 1 prefix-match hit, remaining 8 prompt tokens to eval
llama_perf_context_print:        load time =    9452.77 ms
llama_perf_context_print: prompt eval time =     585.91 ms /     8 tokens (   73.24 ms per token,    13.65 tokens per second)
llama_perf_context_print:        eval time =   31316.61 ms /   149 runs   (  210.18 ms per token,     4.76 tokens per second)
llama_perf_context_print:       total time =   32220.54 ms /   157 tokens


 Black holes are a region in space where the gravitational pull is so strong that nothing, including light, can escape. They are formed when a massive star collapses in on itself and its gravity becomes so strong that it warps the fabric of space and time around it. The point of no return, called the event horizon, marks the boundary of the black hole. Once something crosses the event horizon, it is trapped and cannot escape.
Black holes can range in size from small, stellar-mass black holes formed from the collapse of individual stars, to supermassive black holes found at the centers of galaxies, which can have masses millions or even billions of times that of the sun. The effects of black holes can be seen in the way they distort the


In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "How does photosynthesis work?"}
]

response = llm.create_chat_completion(messages=messages)
print(response["choices"][0]["message"]["content"])


Llama.generate: 1 prefix-match hit, remaining 47 prompt tokens to eval
llama_perf_context_print:        load time =    9452.77 ms
llama_perf_context_print: prompt eval time =    5386.97 ms /    47 tokens (  114.62 ms per token,     8.72 tokens per second)
llama_perf_context_print:        eval time =  101164.98 ms /   463 runs   (  218.50 ms per token,     4.58 tokens per second)
llama_perf_context_print:       total time =  107985.97 ms /   510 tokens


Photosynthesis is the process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of organic compounds, such as glucose. This process is essential for life on Earth, as it provides the energy and organic compounds needed to support the food chain.

Here's a step-by-step explanation of how photosynthesis works:

1. **Light Absorption**: Light is absorbed by pigments such as chlorophyll, which is present in the chloroplasts of plant cells. Chlorophyll absorbs light in the blue and red parts of the visible spectrum, but reflects green light, which is why plants appear green.
2. **Water Absorption**: Water is absorbed by the roots of plants and transported to the chloroplasts, where it is used in the photosynthetic process.
3. **Carbon Dioxide Absorption**: Carbon dioxide is absorbed from the air through small openings on the surface of leaves called stomata.
4. **Light-Dependent Reactions**: Light energy is absorbed by chlorophyll a

In [14]:
for chunk in llm("Explain quantum mechanics", stream=True):
    print(chunk["choices"][0]["text"], end="", flush=True)


Llama.generate: 4 prefix-match hit, remaining 1 prompt tokens to eval


 in a way that's easy to understand
Quantum mechanics is a branch of

llama_perf_context_print:        load time =    9452.77 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    2720.52 ms /    16 runs   (  170.03 ms per token,     5.88 tokens per second)
llama_perf_context_print:       total time =    2754.90 ms /    17 tokens


In [37]:
llm?

Signature:     
llm(
    prompt: 'str',
    suffix: 'Optional[str]' = None,
    max_tokens: 'Optional[int]' = 16,
    temperature: 'float' = 0.8,
    top_p: 'float' = 0.95,
    min_p: 'float' = 0.05,
    typical_p: 'float' = 1.0,
    logprobs: 'Optional[int]' = None,
    echo: 'bool' = False,
    stop: 'Optional[Union[str, List[str]]]' = [],
    frequency_penalty: 'float' = 0.0,
    presence_penalty: 'float' = 0.0,
    repeat_penalty: 'float' = 1.0,
    top_k: 'int' = 40,
    stream: 'bool' = False,
    seed: 'Optional[int]' = None,
    tfs_z: 'float' = 1.0,
    mirostat_mode: 'int' = 0,
    mirostat_tau: 'float' = 5.0,
    mirostat_eta: 'float' = 0.1,
    model: 'Optional[str]' = None,
    stopping_criteria: 'Optional[StoppingCriteriaList]' = None,
    logits_processor: 'Optional[LogitsProcessorList]' = None,
    grammar: 'Optional[LlamaGrammar]' = None,
    logit_bias: 'Optional[Dict[int, float]]' = None,
) -> 'Union[CreateCompletionResponse, Iterator[CreateCompletionStreamResponse]]

In [13]:

augmented_prompt = f"""Using the contexts below, answer the query in detail, write in one para starting with Answer: 

Contexts:
{context}

Query: {query}"""
augmented_prompt


'Using the contexts below, answer the query in detail, write in one para starting with Answer: \n\nContexts:\nAccenture offers two roles with different salary packages. The Associate Software Engineer role comes with a package of Rs. 4.50 LPA, which includes a fixed pay of Rs. 3.83 LPA, a performance bonus of Rs. 32,500 per annum, a one-time joining bonus of Rs. 25,000, along with additional benefits such as gratuity and insurance. The Advanced Associate Software Engineer role offers a Rs. 6.50 LPA package, with a fixed pay of Rs. 5.41 LPA, a performance-based pay of Rs. 82,500 per performance cycle, and a one-time joining bonus of Rs. 26,500. Employees also receive additional benefits such as free transport, house rent allowance, medical allowance, and health insurance with a Rs. 4 lakh cover, which is an advantage over competitors like TCS, CTS, Infosys, and Wipro.. Accenture is a global professional services company that specializes in digital transformation, cloud computing, artifi

In [14]:
response = llm(
    augmented_prompt,
    max_tokens=150,       # Maximum number of tokens to generate
    temperature=0.7,      # Higher = more random, lower = more deterministic
    top_k=50,             # Consider top 50 most likely tokens
    top_p=0.9,            # Nucleus sampling (higher = more diverse output)
    echo=False            # Do not repeat the input in output
)

print(response["choices"][0]["text"])

llama_perf_context_print:        load time =   40165.99 ms
llama_perf_context_print: prompt eval time =   40165.35 ms /   442 tokens (   90.87 ms per token,    11.00 tokens per second)
llama_perf_context_print:        eval time =   21506.06 ms /    69 runs   (  311.68 ms per token,     3.21 tokens per second)
llama_perf_context_print:       total time =   61931.74 ms /   511 tokens



Answer: Accenture offers two distinct roles with different salary packages, catering to the needs of various professionals. The Associate Software Engineer role comes with a package of Rs. 4.50 LPA, which includes a fixed pay of Rs. 3.83 LPA, a performance bonus of Rs. 32,500 per annum, a


In [22]:
query="what are the roles offered by realpage and razor pay"
if latest_org and  len(extract_organization_spacy(query))==1:
    query=latest_org+": "+query
    augmented_prompt=f"""
    rewrite the below question and return the question
    {query}
    """
    response=llm(augmented_prompt)["choices"][0]["text"]
elif extract_organization_spacy(query):
    query=split_mutli_companies(query)
extract_organization_spacy(query)
query


['realpage', 'razor pay']
['realpage', 'razor pay']


Llama.generate: 1 prefix-match hit, remaining 77 prompt tokens to eval
llama_perf_context_print:        load time =    7435.87 ms
llama_perf_context_print: prompt eval time =    5244.46 ms /    77 tokens (   68.11 ms per token,    14.68 tokens per second)
llama_perf_context_print:        eval time =    4917.47 ms /    29 runs   (  169.57 ms per token,     5.90 tokens per second)
llama_perf_context_print:       total time =   10203.35 ms /   106 tokens


[]


' answer: \n     1) roles offered by realpage\n     2) roles offered by razor pay\n        \n        for each question we can have'

In [56]:
augmented_prompt=f"""
    rewrite the below question and return the question
    {query}
"""
response=llm(augmented_prompt)
print(response["choices"][0]["text"])

Llama.generate: 24 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   13658.71 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    2605.97 ms /    16 runs   (  162.87 ms per token,     6.14 tokens per second)
llama_perf_context_print:       total time =    2625.75 ms /    17 tokens


    return the question
    '''
    What is the package offered by modak


In [21]:
def split_mutli_companies(query:str):
    
    prompt=f"""
        for example :
        question: what is the difference between package of accenture and modak
        answer:
        1)what is the package of accenture
        2)what is the package of modak
        split this question in mutliple question based on companies 
        
        question: {query}
        
    """
    response=llm(prompt,max_tokens=30)
    return response["choices"][0]["text"]

In [116]:
split_mutli_companies("what are the roles offered by realpage and razor pay")

Llama.generate: 77 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   13658.71 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    5310.54 ms /    30 runs   (  177.02 ms per token,     5.65 tokens per second)
llama_perf_context_print:       total time =    5349.52 ms /    31 tokens


' answer:\n     1) what are the roles offered by realpage\n     2) what are the roles offered by razor pay\n     split'

In [7]:
query="packages of SDE roles"


In [8]:
def get_response(query,chances=None):
    c=chances
    try:
        query = query
        query_embedding = pc.inference.embed(
            model="multilingual-e5-large",
            inputs=[query],
            parameters={
                "input_type": "query"
            }
        )
        res=[]
        for i in range(7):
            results = index.query(
            namespace=f"example{i}",
            vector=query_embedding[0].values,
            top_k=3,
            include_values=False,
            include_metadata=True
            )
            res.append(results)
            
        # print(results['matches'])
        doc=[]
        for j in res:
            for i in j['matches']:
                doc.append({"id":i['id'],"source_text":i['metadata']['source_text']})
        ranked_results = pc.inference.rerank(
            model="bge-reranker-v2-m3",
            query=query,
            documents=doc,
            top_n=5,
            rank_fields=["source_text"],
            return_documents=True,
            parameters={
                "truncate": "END"
            }
        )
        # print(str(ranked_results))
        con={}
        context=""
        for i in ranked_results.data:
            if i.score>=0.25:
                con[i['document']['source_text']]=i.score
        sorted_dict = []
        for key in sorted(con, key=con.get):
            sorted_dict.append(key)
        for i in sorted_dict[:2]:
            context+=i+". "   
        print(len(context.split()))      
        augmented_prompt = f"""Using the contexts below, answer the query in detail, write in one para, response should strictly start with Answer: 

        Contexts:
        {context[:320]}

        Query: {query}"""
        augmented_prompt

        response = llm(
            augmented_prompt,
            max_tokens=150,       # Maximum number of tokens to generate
            temperature=0.7,      # Higher = more random, lower = more deterministic
            top_k=50,             # Consider top 50 most likely tokens
            top_p=0.9,            # Nucleus sampling (higher = more diverse output)
            echo=False            # Do not repeat the input in output
        )
        return response["choices"][0]["text"].split("Answer:")[1]
    except ValueError as err:
        if c:
            return get_response(chances=c-1,query=query)
        else:
            return err.args

In [9]:
ans=get_response(query=query,chances=3)

215


Llama.generate: 99 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    9784.48 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =   31242.28 ms /   150 runs   (  208.28 ms per token,     4.80 tokens per second)
llama_perf_context_print:       total time =   31551.54 ms /   151 tokens


In [ ]:
ans

' Align Technology offers a competitive salary package for the Software Development Engineer (SDE) role, which includes a fixed pay, performance-based bonuses, and additional benefits such as health insurance, retirement plans, and paid time off, with an average annual salary ranging from Rs. 10-12 LPA. The company provides a comprehensive benefits package to its employees, making it an attractive option for SDEs in the industry. The compensation package not only meets the industry standards but also offers a range of perks and benefits that contribute to the overall well-being of the employees. The average annual salary of Rs. 10-12 LPA is a significant aspect of the package, providing a stable financial foundation for the employees. Furthermore'

215


Llama.generate: 99 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    9784.48 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =   48589.19 ms /   150 runs   (  323.93 ms per token,     3.09 tokens per second)
llama_perf_context_print:       total time =   49167.18 ms /   151 tokens
127.0.0.1 - - [18/Apr/2025 17:40:18] "POST /items HTTP/1.1" 200 -


0


Llama.generate: 29 prefix-match hit, remaining 13 prompt tokens to eval
llama_perf_context_print:        load time =    9784.48 ms
llama_perf_context_print: prompt eval time =    1138.46 ms /    13 tokens (   87.57 ms per token,    11.42 tokens per second)
llama_perf_context_print:        eval time =   33475.81 ms /   149 runs   (  224.67 ms per token,     4.45 tokens per second)
llama_perf_context_print:       total time =   34903.58 ms /   162 tokens
[2025-04-18 17:42:45,053] ERROR in app: Exception on /items [POST]
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\flask\app.py", line 880, in full_d

...

Llama.generate: 41 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    9784.48 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =   30519.24 ms /   150 runs   (  203.46 ms per token,     4.91 tokens per second)
llama_perf_context_print:       total time =   30796.12 ms /   151 tokens
127.0.0.1 - - [18/Apr/2025 17:43:36] "POST /items HTTP/1.1" 200 -


724


Llama.generate: 29 prefix-match hit, remaining 72 prompt tokens to eval
llama_perf_context_print:        load time =    9784.48 ms
llama_perf_context_print: prompt eval time =    6278.12 ms /    72 tokens (   87.20 ms per token,    11.47 tokens per second)
llama_perf_context_print:        eval time =   30884.81 ms /   149 runs   (  207.28 ms per token,     4.82 tokens per second)
llama_perf_context_print:       total time =   37457.47 ms /   221 tokens
127.0.0.1 - - [18/Apr/2025 17:45:58] "POST /items HTTP/1.1" 200 -


105


Llama.generate: 30 prefix-match hit, remaining 78 prompt tokens to eval
llama_perf_context_print:        load time =    9784.48 ms
llama_perf_context_print: prompt eval time =    6955.07 ms /    78 tokens (   89.17 ms per token,    11.21 tokens per second)
llama_perf_context_print:        eval time =   31634.42 ms /   149 runs   (  212.31 ms per token,     4.71 tokens per second)
llama_perf_context_print:       total time =   38888.14 ms /   227 tokens
127.0.0.1 - - [18/Apr/2025 17:48:17] "POST /items HTTP/1.1" 200 -


In [ ]:
from flask import Flask, jsonify, request
import nest_asyncio
from threading import Thread

nest_asyncio.apply()


app = Flask(__name__)


chats = []  # Separate chat data

@app.route('/')
def home():
    return jsonify({"message": "Welcome to the Flask API!"})

@app.route('/items', methods=['GET'])
def get_items():
    return jsonify(items)

@app.route('/items', methods=['POST'])
def add_item():
    data = request.get_json()
    query=data['query']
    res=get_response(query=query,chances=3)
    return jsonify(res)


def run_app():
    app.run(port=5000, debug=False)

thread = Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


157


llama_perf_context_print:        load time =    9784.48 ms
llama_perf_context_print: prompt eval time =    9783.66 ms /    98 tokens (   99.83 ms per token,    10.02 tokens per second)
llama_perf_context_print:        eval time =   46758.16 ms /   149 runs   (  313.81 ms per token,     3.19 tokens per second)
llama_perf_context_print:       total time =   57123.77 ms /   247 tokens
127.0.0.1 - - [18/Apr/2025 17:27:08] "POST /items HTTP/1.1" 200 -


386


...

Llama.generate: 86 prefix-match hit, remaining 7 prompt tokens to eval
llama_perf_context_print:        load time =    9784.48 ms
llama_perf_context_print: prompt eval time =     589.46 ms /     7 tokens (   84.21 ms per token,    11.88 tokens per second)
llama_perf_context_print:        eval time =   32347.60 ms /   149 runs   (  217.10 ms per token,     4.61 tokens per second)
llama_perf_context_print:       total time =   33214.53 ms /   156 tokens
127.0.0.1 - - [18/Apr/2025 17:32:26] "POST /items HTTP/1.1" 200 -


386


Llama.generate: 90 prefix-match hit, remaining 4 prompt tokens to eval
llama_perf_context_print:        load time =    9784.48 ms
llama_perf_context_print: prompt eval time =     358.39 ms /     4 tokens (   89.60 ms per token,    11.16 tokens per second)
llama_perf_context_print:        eval time =   30476.51 ms /   149 runs   (  204.54 ms per token,     4.89 tokens per second)
llama_perf_context_print:       total time =   31090.52 ms /   153 tokens
127.0.0.1 - - [18/Apr/2025 17:34:39] "POST /items HTTP/1.1" 200 -


215


Llama.generate: 30 prefix-match hit, remaining 70 prompt tokens to eval
llama_perf_context_print:        load time =    9784.48 ms
llama_perf_context_print: prompt eval time =   10388.73 ms /    70 tokens (  148.41 ms per token,     6.74 tokens per second)
llama_perf_context_print:        eval time =   53201.26 ms /   149 runs   (  357.06 ms per token,     2.80 tokens per second)
llama_perf_context_print:       total time =   64166.96 ms /   219 tokens
[2025-04-18 17:38:00,592] ERROR in app: Exception on /items [POST]
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\flask\app.py", line 880, in full_d

In [41]:
!netstat -ano | findstr :5000


  TCP    127.0.0.1:5000         0.0.0.0:0              LISTENING       19188


In [45]:
!taskkill /PID 19188 /F


: 